<a href="https://colab.research.google.com/github/amoladmane/Deep-Learning/blob/main/Regression_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regression with Keras

## Step 1 - read data set

In [2]:
import pandas as pd
url = "https://raw.githubusercontent.com/utkarshg1/Neural-Net-Datasets/main/PowerPlant.csv"
df = pd.read_csv(url)
df.head()

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


## target feature is PE ( Power Plant Energy Output)

## Step2 - basic data quality check


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [4]:
df.isna().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [5]:
df.duplicated().sum()

41

In [6]:
df.duplicated().sum()/len(df)

0.004285117056856187

## Step 3 - seprate X and Y ( PE )

In [7]:
X = df.drop(columns =['PE'])
Y = df[['PE']]

In [8]:
X[0:5]

,AT,V,AP,RH
0,8.34,40.77,1010.84,90.01
1,23.64,58.49,1011.40,74.20
2,29.74,56.90,1007.15,41.91
3,19.07,49.69,1007.22,76.79
4,11.80,40.66,1017.13,97.20


In [9]:
Y.head()

,PE
0,480.48
1,445.75
2,438.76
3,453.09
4,464.43


## Step 4 - Preprocissing ( scaling data )

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler


In [11]:
num_pipe = Pipeline(steps=[('impute', SimpleImputer(strategy='mean')),
                           ('scaler',StandardScaler())]).set_output(transform='pandas')

In [12]:
X_pre = num_pipe.fit_transform(X)
X_pre.head()

,AT,V,AP,RH
0,-1.517862,-1.065205,-0.407357,1.143944
1,0.535255,0.329277,-0.313057,0.061031
2,1.353818,0.204151,-1.028729,-2.150688
3,-0.077996,-0.363242,-1.016941,0.238434
4,-1.053562,-1.073861,0.651838,1.636427


## Step 5 - Apply train test split
20% data seen to model

In [13]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X_pre, Y, test_size=0.20, random_state=22)

In [14]:
xtrain.shape

(7654, 4)

In [15]:
xtest.shape

(1914, 4)

In [16]:
ytest.head()

,PE
4938,459.48
2807,438.23
8607,465.56
3998,428.62
6597,435.70


## Step 5 - Build Keras Neural Network

In [17]:
from keras.models import Sequential

In [18]:
from keras.layers import Dense, Input

In [19]:
# initilise the sequential model
model = Sequential()
# add input layer
model.add(Input(shape=(xtrain.shape[1],)))
# add hidden layer 1
model.add(Dense(units=8, activation='relu'))
# add hidden layer 2
model.add(Dense(units=4,activation='relu'))
# add output layer
model.add(Dense(units=1, activation='linear'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 40        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 1)                 5         
                                                                 
Total params: 81 (324.00 Byte)
Trainable params: 81 (324.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])



In [24]:
hist=model.fit(xtrain,ytrain, validation_split=0.20, epochs=100,)

Epoch 1/100
192/192 [==============================] - 4s 5ms/step - loss: 206409.9375 - mae: 453.9913 - val_loss: 204280.3750 - val_mae: 451.6472
Epoch 2/100
192/192 [==============================] - 1s 4ms/step - loss: 201593.4219 - mae: 448.5595 - val_loss: 194504.4062 - val_mae: 440.4589
Epoch 3/100
192/192 [==============================] - 1s 4ms/step - loss: 184068.6250 - mae: 427.7869 - val_loss: 167144.5938 - val_mae: 406.5446
Epoch 4/100
192/192 [==============================] - 1s 4ms/step - loss: 147447.7188 - mae: 378.4088 - val_loss: 122021.7344 - val_mae: 339.4395
Epoch 5/100
192/192 [==============================] - 1s 5ms/step - loss: 100636.9297 - mae: 300.0580 - val_loss: 77964.3594 - val_mae: 256.0743
Epoch 6/100
192/192 [==============================] - 1s 5ms/step - loss: 63055.8672 - mae: 228.0628 - val_loss: 48602.8164 - val_mae: 199.1326
Epoch 7/100
192/192 [==============================] - 1s 6ms/step - loss: 38239.1680 - mae: 175.8647 - val_loss: 29320.5

## Evaluate Model

In [25]:
model.evaluate(xtrain,ytrain)

240/240 [==============================] - 1s 4ms/step - loss: 20.0106 - mae: 3.5338


[20.010602951049805, 3.5338170528411865]

In [26]:
model.evaluate(xtest,ytest)

60/60 [==============================] - 0s 4ms/step - loss: 19.7239 - mae: 3.5635


[19.72391700744629, 3.563546895980835]